In [2]:
cd /Users/karolinegriesbach/Documents/Innkeepr/Git/evaluation-and-execution-scripts/

In [44]:
import logging
import pandas as pd
import numpy as np
import delta_sharing
import awswrangler as wr

import general_functions.databricks_client as db_client
from general_functions.return_account_ids import return_account_ids
from general_functions.constants import return_api_url
from general_functions.call_api_with_account_id import call_api_with_accountId, send_to_innkeepr_api_paginated

In [46]:
customer = "Asambeauty"
min_session_date = "2025-07-15"
max_session_date = "2025-07-22"

audience = "685e571534f2e2097f339194"
url = return_api_url()
print(f"url = {url}")
account_id = return_account_ids()
account_id = [acc["id"] for acc in account_id if acc["name"] == customer]
account_id = account_id[0]
criteo_treatments = ['686f0369d3c5570fc4690551',
  '687054e9d3c5574f786913b1',
  '687054e9d3c557c0817ecf07',
  '686f0369d3c5570fce14b44b',
  '686f0369d3c5570fd1028a40',
  '686f0368d3c5570fd7437ab4',
  '686f0368d3c5570fd36ead2f',
  '686f0368d3c5570fd05e2124',
  '686f0369d3c5570fc4690550',
  '686f0369d3c5570fd05e2126']

In [64]:
profile_path = db_client.return_databricks_client()
table_path = f"{profile_path}#delta_share_events.{account_id}.features_view_30_outlook"
df_all = delta_sharing.load_as_pandas(table_path, limit=1000000)

In [65]:
#df = pd.read_csv(f"SprintStories/XXX-Asambeauty-Criteo-Analyse/{customer}_features_view_30_outlook.csv", index=False)

use_columns = ["created","session","anonymousId","url","conv_name"]
df = df_all[use_columns]
#df.to_csv(f"SprintStories/XXX-Asambeauty-Criteo-Analyse/{customer}_features_view_30_outlook.csv", index=False)
    

In [66]:
treatments = send_to_innkeepr_api_paginated(
    f"{url}/treatments/query",
    account_id,
    {"id":criteo_treatments},
    logging
)
treatments = pd.json_normalize(treatments)

In [67]:
externalIds = treatments["externalId"].unique().tolist()
externalIds

In [68]:
sessions_with_checkout_completed=df[df["conv_name"]=="checkout_completed"]["session"].unique().tolist()
df["target"] = np.where(df["session"].isin(sessions_with_checkout_completed), "checkout_completed", "no_conversion")

In [69]:
criteo_data = df[df["url"].astype("string").str.contains("utm_source=criteo")]
criteo_data["utm_id"] = criteo_data["url"].astype("string").str.split("utm_id=").str[1].str.split("&").str[0]
criteo_data = pd.merge(criteo_data, treatments[["relates_to.campaign.name","id","externalId","name"]], how="left", left_on="utm_id", right_on="externalId")
criteo_data = criteo_data[criteo_data["externalId"].isin(externalIds)]
criteo_data

In [70]:
date_range = pd.date_range(min_session_date, max_session_date, freq="D").strftime("%Y%m%d").tolist()
targeting_history = pd.DataFrame()
for date in date_range:
    path = f"s3://{account_id}/targeting.history/{date}/{audience}.parquet"
    try:
        temp = wr.s3.read_parquet(path)
    except wr.exceptions.NoFilesFound:
        print(f"No file found for {date}")
        continue
    temp["s3_date"] = date
    targeting_history = pd.concat([targeting_history, temp])
targeting_history

In [71]:
criteo_sessions = criteo_data["session"].unique().tolist()
conversions = df.drop_duplicates(subset=["session","target"])[["session","target"]].reset_index(drop=True)
targeting_history["criteo"] = np.where(targeting_history["session"].isin(criteo_sessions), "criteo", "no_criteo")
targeting_history = pd.merge(targeting_history, conversions, how="left", left_on="session", right_on="session")
targeting_history

In [72]:
targeting_history.groupby(by=["criteo","target"])["conv_prob"].describe().reset_index()

In [73]:
converted_anonymousIds = targeting_history[targeting_history["target"]=="checkout_completed"]["anonymousId"].unique().tolist()
targeting_history["target_anonymousId"] = np.where(targeting_history["anonymousId"].isin(converted_anonymousIds), "checkout_completed", "no_conversion")
targeting_history.groupby(by=["criteo","target_anonymousId"])["conv_prob"].describe().reset_index()

In [74]:
targeting_history.groupby(by=["criteo"])["conv_prob"].describe().reset_index()